In [1]:
#Upload Dataset via Code
from google.colab import files

uploaded = files.upload()


Saving emails.csv to emails.csv


In [2]:
#Import Libraries
import pandas as pd
import nltk
import string

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [22]:
#Download Stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
#Load Dataset Safely
file_name = list(uploaded.keys())[0]
data = pd.read_csv(file_name)

print(data.head())
print(data.columns)
#Make sure your dataset has:

##one column with text

#one column with path or identifier

                       file                                            message
0     allen-p/_sent_mail/1.  Message-ID: <18782981.1075855378110.JavaMail.e...
1    allen-p/_sent_mail/10.  Message-ID: <15464986.1075855378456.JavaMail.e...
2   allen-p/_sent_mail/100.  Message-ID: <24216240.1075855687451.JavaMail.e...
3  allen-p/_sent_mail/1000.  Message-ID: <13505866.1075863688222.JavaMail.e...
4  allen-p/_sent_mail/1001.  Message-ID: <30922949.1075863688243.JavaMail.e...
Index(['file', 'message'], dtype='object')


In [14]:
#Extract Labels from File Path (FIX)
def extract_label(path):
    path = str(path).lower()
    if "spam" in path:
        return 1
    else:
        return 0

data["label"] = data["file"].apply(extract_label)

In [15]:
print(data["label"].value_counts())
#This guarantees NO NaN values

label
0    517401
Name: count, dtype: int64


In [16]:
#Text Preprocessing
stop_words = set(stopwords.words("english"))

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans("", "", string.punctuation))
    words = text.split()
    words = [w for w in words if w not in stop_words]
    return " ".join(words)

data["message"] = data["message"].apply(preprocess_text)


In [17]:

#TF-IDF Vectorization
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data["message"])
y = data["label"]


In [18]:
#Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [19]:
#Train Model
model = MultinomialNB()
model.fit(X_train, y_train)


MultinomialNB()

In [20]:
#Evaluate Model
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))


Accuracy: 1.0
Precision: 0.0
Recall: 0.0
F1 Score: 0.0


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [21]:
#Custom Prediction
def predict_spam(message):
    message = preprocess_text(message)
    vector = vectorizer.transform([message])
    return "Spam" if model.predict(vector)[0] == 1 else "Ham"

print(predict_spam("You won a free lottery prize"))
print(predict_spam("Let's meet after class"))


Ham
Ham
